In [1]:
# Attempt to train an mnist model and distill it into a smaller model
# conclusion, distillation is a scam, at least for mnist

In [2]:
%pylab inline
import os
import time
import random
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = [x/255.0 for x in [x_train, x_test]]
y_train, y_test = [tf.keras.utils.to_categorical(x) for x in [y_train, y_test]]

Populating the interactive namespace from numpy and matplotlib


In [3]:
def get_m(c1=True, c2=True, loss='categorical_crossentropy'):
    x = in1 = Input((28,28))
    x = Reshape((28,28,1))(x)

    if c1:
        x = Conv2D(64, (5,5), padding='same', strides=(2,2))(x)
        x = ELU()(x)

    if c2:
        x = Conv2D(128, (5,5), padding='same', strides=(2,2))(x)
        x = ELU()(x)

    x = Flatten()(x)
    x = Dense(128)(x)
    x = ELU()(x)
    x = Dense(10, activation='sigmoid')(x)

    m = Model(in1, x)
    m.compile('adam', loss, metrics=['accuracy'])
    return m

In [ ]:
m = get_m()
m.summary()
m.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=128, epochs=3)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 14, 14, 64)        1664      
_________________________________________________________________
elu (ELU)                    (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
elu_1 (ELU)                  (None, 7, 7, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0     

In [ ]:
rv = m.predict(x_train, verbose=1)

In [ ]:
sm1 = get_m(False, False, 'categorical_crossentropy')
sm2 = get_m(False, False, 'categorical_crossentropy')
sm1.summary()

In [ ]:
sm1.fit(x_train, rv, validation_data=(x_test, y_test), batch_size=128, epochs=5)

In [ ]:
sm2.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=128, epochs=5)